In [14]:
import pandas as pd
from sklearn.svm import LinearSVC
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation

In [2]:
data = pd.read_csv('../datasets/complete-set.csv')

In [3]:
del data['Unnamed: 0']
data.dropna()

,content,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1
...,...,...
33795,"For the first time in more than a decade, impo...",0
33796,Says Donald Trump has bankrupted his companies...,0
33797,"John McCain and George Bush have ""absolutely n...",0
33798,A new poll shows 62 percent support the presid...,1


In [4]:
MAX_NB_WORDS = 100000 # max number of words for tokenizer
MAX_SEQUENCE_LENGTH = 10000 # max length of each sentences, including padding
VALIDATION_SPLIT = 0.2 # 20% of data for validation (not used in training)
EMBEDDING_DIM = 300 # embedding dimensions for word vectors
GLOVE_DIR = '/home/till/2.Sem/euvsvirus/models/glove.840B.300d.txt'   # "glove840B."+str(EMBEDDING_DIM)+"d.txt"

In [5]:
texts = data['content'].to_numpy()

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))

Vocabulary size: 240108


In [7]:
def tree_to_glove(train_set, embeddings_index):
    x_list = []
    x_array = []

    for sentence in train_set:
        for word in sentence:
            if embeddings_index.get(str(word)) is not None:
                if len(embeddings_index.get(str(word))):
                    x = embeddings_index.get(str(word))
                    x_list.append(x)
        x_array.append(x_list)
        x_list = []
    return x_array

In [8]:
embeddings_index = {}

f = open(GLOVE_DIR, encoding='utf8')
print('Loading Glove from:', GLOVE_DIR,'…', end='')
for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

#for line in f:
#    values = line.split()
#    word = values[0]
#    embeddings_index[word] = np.asarray(values[1:], dtype='float64')

f.close()
print("Done.\n Proceeding with Embedding Matrix…", end="")

Loading Glove from: /home/till/2.Sem/euvsvirus/models/glove.840B.300d.txt …

/home/till/anaconda3/envs/tflow/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  import sys


Done.
 Proceeding with Embedding Matrix…

In [9]:
#clean_data = tree_to_glove(sequences, embeddings_index)

text_data = pad_sequences(sequences, padding = 'post', maxlen = MAX_SEQUENCE_LENGTH, dtype='float64')
labels = data['label'].to_numpy()
print('Shape of data tensor:', text_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (33800, 10000)
Shape of label tensor: (33800,)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(text_data,labels)
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
print("Completed!")

Completed!


In [11]:
svm = LinearSVC()
svm.fit(x_train,y_train)
svm.score(x_test,y_test)

/home/till/anaconda3/envs/tflow/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.493491124260355

In [19]:
model = Sequential([Dense(512, input_shape=(10000,)), Activation('relu'),
                   Dense(128), Activation('relu'),
                   Dense(1), Activation('softmax')])

In [20]:
model.compile(loss='mean_squared_error',optimizer='SGD')

In [21]:
model.fit(x_train,y_train)

Epoch 1/1
25350/25350 [==============================] - 47s 2ms/step - loss: 0.5218
